<a href="https://colab.research.google.com/github/sanAkel/ufs_diurnal_diagnostics/blob/main/HAFS/hafs_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install boto3
!pip install botocore
!pip install xarray
!pip install matplotlib
!pip install xmovie

!apt-get update && apt-get install -y ffmpeg netcdf4
!pip install netcdf4

In [ ]:
import boto3
import botocore

import xarray as xr
from xmovie import Movie

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def download_s3_file(BUCKET_NAME, KEY, fname):
    """
    Downloads a file from an AWS S3 bucket.

    Args:
        BUCKET_NAME (str): The name of the S3 bucket.
        KEY (str): The key (prefix) of the object in the bucket.
        fname (str): The filename of the object to download.

    Returns:
        str or None: The local filename if successful, None otherwise.
    """
    # KEY_WITH_FNAME = f'{KEY}{fname}' # Construct the full key with the filename - This was incorrect

    s3 = boto3.client('s3', config=botocore.config.Config(signature_version=botocore.UNSIGNED))

    try:
        s3.download_file(BUCKET_NAME, f'{KEY}/{fname}', fname) # Use the constructed key and fname for local filename
        print(f"Successfully downloaded {KEY}/{fname} to {fname}")
        return fname
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print(f"The object does not exist: {KEY}/{fname}")
        else:
            print(f"An error occurred: {e}")
        return None

# Inputs

In [ ]:
BUCKET_NAME = 'noaa-nws-hafs-pds'

forecast_start_date = '20251026' # yyyymmdd
forecast_hour = '00' # 4-synoptic times

basin = "13l"
which_hafs = "hfsa"
model = "mom6"


KEY = f'{which_hafs}/{forecast_start_date}/{forecast_hour}'
hr_start, hr_end, freq = [0, 123, 3]

# domain indices
x_start, x_end = [-90, -55]
y_start, y_end = [10, 25]

# Jamaica? In x- and y- indices! The nc file is POORLY constructed.
x_pos = 310
y_pos_start, y_pos_end = [200, 250]

## Get the data

In [ ]:
filenames = [f'{basin}.{forecast_start_date}{forecast_hour}.{which_hafs}.{model}.f{hour:03d}.nc' for hour in range(hr_start, hr_end, freq)]

for fname in filenames:
  #print(f'Downloading: {fname}')
  download_s3_file(BUCKET_NAME, KEY, fname)
  ds = xr.open_dataset(fname)
  ds_cull = ds[['temp', 'so']]
  ds_subset = ds_cull.sel(xh=slice(x_start, x_end), yh=slice(y_start, y_end))
  fname_subset = fname.replace('.nc', '_subset.nc')
  ds_subset.to_netcdf(fname_subset)
  !rm {fname}

In [ ]:
subset_filenames = [fname.replace('.nc', '_subset.nc') for fname in filenames]
ds = xr.open_mfdataset(subset_filenames)

In [ ]:
movie = Movie(ds.isel(z_l=0).temp,
              framedim='time',
              input_check=False,
              x='xh',
              y='yh',
              vmin=25,
              vmax=32,
              cmap='Spectral_r')

In [ ]:
movie.save('temperature_animation_xmovie.gif')

In [ ]:
movie = Movie(ds.isel(z_l=0).so,
              framedim='time',
              input_check=False,
              x='xh',
              y='yh',
              vmin=34,
              vmax=38,
              cmap='jet')

In [ ]:
movie.save('salinity_animation_xmovie.gif')